# Data Processing

In [ ]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)

In [37]:
# Define la ruta a la carpeta donde se guardan los archivos
output_dir = "../data/scrapped"

try:
    # 1. Lista todos los archivos en la carpeta
    all_files = os.listdir(output_dir)

    # 2. Filtra solo los archivos que terminan en '.parquet'
    parquet_files = [os.path.join(output_dir, f) for f in all_files if f.endswith('.parquet')]

    # 3. Verifica si hay archivos para procesar
    if not parquet_files:
        print("No se encontraron archivos .parquet en la carpeta especificada.")
    else:
        print(f"Se encontraron {len(parquet_files)} archivos .parquet. Concatenando...")

        # 4. Lee cada archivo Parquet en un DataFrame y los guarda en una lista
        list_of_dfs = [pd.read_parquet(f) for f in parquet_files]

        # 5. Concatena todos los DataFrames de la lista
        combined_df = pd.concat(list_of_dfs, ignore_index=True)

        print("¡Concatenación exitosa! Los archivos se han unido en un solo DataFrame.")
        print("Se ha creado un DataFrame con las siguientes dimensiones:")
        print(f"Filas: {combined_df.shape[0]}, Columnas: {combined_df.shape[1]}")

except FileNotFoundError:
    print("Error: No se encontró la carpeta especificada.")
    print(f"Por favor, revisa que la carpeta '{output_dir}' exista.")
except ImportError:
    print("Error: La librería 'pyarrow' o 'fastparquet' no está instalada.")
    print("Para leer archivos .parquet, necesitas instalar una de estas librerías. Puedes usar el siguiente comando: pip install pyarrow")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Se encontraron 378 archivos .parquet. Concatenando...
¡Concatenación exitosa! Los archivos se han unido en un solo DataFrame.
Se ha creado un DataFrame con las siguientes dimensiones:
Filas: 9033, Columnas: 35


## Tipos de datos

In [38]:
combined_df.head(3)

,IdJugador,IdClub,IdEquipo,Nombre,NombreCompleto,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos,plus_minus,posesiones_consumidas,posesiones_jugadas,rebote_of_disp,rebote_def_disp,puntos_q4_y_prorroga,puntos_clutch,posesiones_estimadas,partido_key
0,96264,1932,0,"ARIAS, J.","ARIAS, JUAN PABLO",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,ARGENTINO (J),0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00)
1,96269,1932,0,"CAPELLI, S.","CAPELLI, SANTIAGO",0,1,0,1,1,0,0,0,0,1,0,1,06:05,False,ARGENTINO (J),0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00)
2,323790,1932,0,"FRONTERA, R.","FRONTERA, RAMIRO JEREMIAS",5,6,0,6,2,1,1,1,2,3,2,3,24:02,False,ARGENTINO (J),2,0,0,0,1,0,-21,11,48,30,15,0,0,3.44,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00)


In [39]:
# Convertir la columna 'TiempoJuego' de string "MM:SS" a segundos como entero
def tiempo_a_segundos(tiempo_str):
    try:
        minutos, segundos = map(int, tiempo_str.split(":"))
        return minutos * 60 + segundos
    except Exception:
        return 0

combined_df["TiempoJuego_seg"] = combined_df["TiempoJuego"].apply(tiempo_a_segundos)
combined_df["TiempoJuego_min"] = combined_df["TiempoJuego_seg"] / 60

In [40]:
categorical_cols=['IdJugador', 'IdClub', 'IdEquipo','Nombre', 'NombreCompleto', 'equipo', 'partido_key']
numerical_cols=['Puntos', 'ReboteDefensivo','ReboteOfensivo', 'RebotesTotales', 'Asistencias', 'Recuperaciones',
                'Perdidas', 'TaponCometido', 'TaponRecibido', 'FaltaCometida','FaltaRecibida', 'Valoracion', 
                'TirosDosAciertos', 'TirosDosFallos','TirosTresAciertos', 'TirosTresFallos', 'TirosLibresAciertos',
                'TirosLibresFallos', 'plus_minus', 'posesiones_consumidas','posesiones_jugadas', 'rebote_of_disp', 
                'rebote_def_disp','puntos_q4_y_prorroga', 'puntos_clutch', 'posesiones_estimadas','TiempoJuego_seg','TiempoJuego_min']

In [41]:
# Convertir columnas categóricas a 'category' y numéricas a 'float' (o 'int' si no hay NaN)
for col in categorical_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype('category')

for col in numerical_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype(float)


## Group By

In [42]:
df_players = combined_df.groupby("NombreCompleto", observed=False).agg(
    {col: "sum" if pd.api.types.is_numeric_dtype(combined_df[col]) else "last" for col in combined_df.columns if col != "NombreCompleto"}
).reset_index()

# Agregar la columna CantidadPartidosJugados
cantidad_partidos = combined_df[combined_df["TiempoJuego_seg"] > 0].groupby("NombreCompleto", observed=False).size()
df_players["CantidadPartidosJugados"] = df_players["NombreCompleto"].map(cantidad_partidos).fillna(0).astype(int)

In [43]:
df_players

,NombreCompleto,IdJugador,IdClub,IdEquipo,Nombre,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos,plus_minus,posesiones_consumidas,posesiones_jugadas,rebote_of_disp,rebote_def_disp,puntos_q4_y_prorroga,puntos_clutch,posesiones_estimadas,partido_key,TiempoJuego_seg,TiempoJuego_min,CantidadPartidosJugados
0,"AALIYA, LEE ABRAHAM",326138,1790,0,"AALIYA, L.",261.0,82.0,42.0,124.0,18.0,18.0,25.0,25.0,4.0,60.0,51.0,252.0,17:57,1,INSTITUTO,79.0,0.0,23.0,0.0,34.0,0.0,98.0,278.0,1508.0,663.0,683.0,104.0,2.0,99.96,UNION (SF) vs INSTITUTO (005/12/2024 21:00),37472.0,624.533333,34
1,"ACEVEDO, MAXIMILIANO JUNIORS",149814,2305,0,"ACEVEDO, M.",2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,00:00,0,OBERA,1.0,0.0,0.0,0.0,0.0,0.0,-6.0,4.0,26.0,15.0,10.0,2.0,0.0,0.00,ZARATE BASKET vs OBERA (029/01/2025 22:00),694.0,11.566667,4
2,"ACEVEDO, SEBASTIAN IGNACIO",209965,1426,0,"ACEVEDO, S.",285.0,104.0,24.0,128.0,24.0,26.0,30.0,1.0,5.0,84.0,65.0,266.0,18:08,14,SAN MARTIN (C),55.0,0.0,43.0,0.0,46.0,0.0,-12.0,287.0,2047.0,992.0,828.0,64.0,12.0,124.24,ZARATE BASKET vs SAN MARTIN (C) (003/04/2025 2...,50524.0,842.066667,38
3,"ACTIS, JOAQUIN MATIAS",225273,88,0,"ACTIS, J.",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,00:00,0,SAN LORENZO,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,1.0,8.0,6.0,3.0,0.0,0.0,1.00,SAN LORENZO vs SAN MARTIN (C) (027/01/2025 20:00),226.0,3.766667,1
4,"ACUÑA, ROBERTO SANTIAGO",330244,2376,0,"ACUÑA, R.",213.0,95.0,49.0,144.0,20.0,16.0,28.0,11.0,10.0,79.0,48.0,238.0,20:04,19,PEÑAROL (MDP),87.0,0.0,2.0,0.0,33.0,0.0,114.0,215.0,1372.0,599.0,597.0,48.0,2.0,87.08,UNION (SF) vs PEÑAROL (MDP) (002/05/2025 21:00),39416.0,656.933333,35
5,"AGUERRE, FEDERICO MATIAS",314345,1903,0,"AGUERRE, F.",308.0,174.0,34.0,208.0,37.0,17.0,32.0,13.0,4.0,69.0,75.0,409.0,21:46,35,RIACHUELO (LR),51.0,0.0,51.0,0.0,53.0,0.0,72.0,295.0,1798.0,700.0,900.0,56.0,8.0,123.32,UNION (SF) vs RIACHUELO (LR) (003/12/2024 21:00),56024.0,933.733333,36
6,"AGUIRRE, GUSTAVO NICOLAS",173453,1424,0,"AGUIRRE, G.",249.0,150.0,20.0,170.0,89.0,31.0,44.0,0.0,3.0,55.0,74.0,351.0,31:43,31,REGATAS (C),53.0,0.0,36.0,0.0,35.0,0.0,16.0,304.0,1975.0,865.0,925.0,42.0,23.0,129.40,UNION (SF) vs REGATAS (C) (031/03/2025 21:00),55567.0,926.116667,34
7,"ALESSIO, JOSE IGNACIO JESUS",217941,1913,0,"ALESSIO, J.",348.0,67.0,18.0,85.0,22.0,6.0,29.0,0.0,8.0,64.0,82.0,322.0,10:27,13,UNION (SF),112.0,0.0,17.0,0.0,73.0,0.0,-138.0,297.0,1296.0,549.0,577.0,85.0,10.0,172.12,ZARATE BASKET vs UNION (SF) (020/12/2024 21:00),36588.0,609.800000,33
8,"ALEXANDER, QUINTIN IMMANUEL",328795,1424,0,"ALEXANDER, Q.",148.0,92.0,24.0,116.0,14.0,9.0,19.0,3.0,5.0,59.0,40.0,181.0,21:43,0,REGATAS (C),63.0,0.0,0.0,0.0,22.0,0.0,-25.0,146.0,826.0,385.0,400.0,33.0,4.0,67.68,ZARATE BASKET vs REGATAS (C) (005/05/2025 21:00),23483.0,391.383333,25
9,"ALI, PRINCE ADAMS",328755,1869,0,"ALI , P.",63.0,10.0,8.0,18.0,7.0,6.0,10.0,1.0,2.0,14.0,22.0,57.0,17:59,4,QUIMSA,16.0,0.0,5.0,0.0,16.0,0.0,-19.0,70.0,202.0,80.0,75.0,12.0,0.0,30.04,UNION (SF) vs QUIMSA (020/01/2025 21:00),6311.0,105.183333,5


## Nacionalidad de Jugadores

In [44]:
extranjeros_data = [
    {"NombreCompleto": "CHACON TIRADO, MARCOS", "nacionalidad": "Cubano"},
    {"NombreCompleto": "MILLER, TAVARIO EARNEST PTRISTIAN", "nacionalidad": "Bahamense"},
    {"NombreCompleto": "RAMIREZ ALCANTARA, KELVIN LEANDRO", "nacionalidad": "Dominicano"},
    {"NombreCompleto": "OWENS, DEMARCO RASHAD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "THOMAS JR, MARCUS WILEY", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "VORHEES, WILLIAM LEONARD-DEUBLER", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "FERGUSON, ROMEAO VENILL", "nacionalidad": "Estadounidense"}, 
    {"NombreCompleto": "LOCKETT, PHILLIP DOMINIQUE DANIEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CLARKE, CHRISTOPHER ASHTON", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "THORNTON, WILLIE ALFORD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "DANIELS, TRAVIS DORREL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "DIGGS, AVERY GERELL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BECTON, REGINALD GEQUAN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CARRERAS PEGUERO, XAVIER MANUEL", "nacionalidad": "Dominicano"},
    {"NombreCompleto": "THOMAS III, CHARLES PRICE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "KRAMER, KELBY JOHN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "GIVENS, SAMUEL JAMAL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BOWIE JR, JULIUS R", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CALFANI PERSINCULA, MATHIAS KENY", "nacionalidad": "Uruguayo"},
    {"NombreCompleto": "MORRISON, DOMINIQUE MONTEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ALEXANDER, QUINTIN IMMANUEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CARDENAS ZAMORA, JORDAN ISRAEL", "nacionalidad": "Ecuatoriano"},
    {"NombreCompleto": "ASCANIO SOLORZANO, JOSE GREGORIO", "nacionalidad": "Venezolano"},
    {"NombreCompleto": "WALTON, ZACHERY CHRISTOPHER", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "THOMAS, DISCHON KYIR", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "PRIDDY, NATHAN WAYNE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "WHITFIELD III, ROBERT JAMARCUS", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HORTON, KENNETH WILCHER", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HOLT, EMMITT DWIGHT", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "JENKINS, JALEN KEMAL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "SANDERS, NAKIE GERALD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BOND JR, TIMOTHY LAMONT", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ROBINSON, BRANDON LAMAR", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "OBENG MENSAH, YAW", "nacionalidad": "Canadiense"},
    {"NombreCompleto": "TROCHA MORELOS, TONNY JOSE", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "CRAION JR, MICHAEL JOE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BELL, RANDY TYREE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "DANIELS, DEANDRE MARTISE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "REESE V, JAMES LANARD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "WATSON, EARL OVREL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ROQUE MARTINEZ, ROMARIO JOSE", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "PAYTON CLOTTEY, EMMANUEL TRAVON", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "FIELDS, CALEB JOSEPH", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ANDERSON, ALPHONSO JORDAN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BASTARDO JOSE, RAYMON SCARLIN", "nacionalidad": "Dominicano"},
    {"NombreCompleto": "BANYARD, NICHOLAS RYAN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "MAXWELL, DU'VAUGHN ELISHA", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "RUIZ RUIZ, JOSE DANIEL", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "FUNDORA ARRECHAVALETA, YASMANY	", "nacionalidad": "Cubano"}, 
    {"NombreCompleto": "KRAYEM, OMAR NABIL", "nacionalidad": "Palestino"}, # Estadounidense
    {"NombreCompleto": "WALLACE, DEVANTE RASHAD-KEITH", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "STOKES, KAMAU THUTMOSES", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "LOWERY, DISHON LURELL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ALI, PRINCE ADAMS", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BONIZIOLI HONORATO, ITALO	", "nacionalidad": "Brasileño"}, 
    {"NombreCompleto": "CARTER, MYLES JUSTIN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HOOPER, CHRISTOPHER JALEEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "OPOKU, NANA KWASI HYEAKURO", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HAMILTON, ISAAC BRANDON", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "NAYLOR, CAMERON LEVELE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "MADRIGAL RENTERIA, JUAN ESTEBAN", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "NIEBLES HERRERA, EDWIN JOSE", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "PETRI, NICHOLAS", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "GUERRA CAÑATE, YEFERSON ANTONIO", "nacionalidad": "Venezolano"},
    {"NombreCompleto": "GARCIA GUERRERO, JORGE LUIS", "nacionalidad": "Venezolano"}, 
    {"NombreCompleto": "HERNANDEZ, MANUEL ALONSO", "nacionalidad": "Estadounidense"}, # Mexicano
]

df_extranjeros = pd.DataFrame(extranjeros_data)
df_extranjeros['nacionalidad'].value_counts()

nacionalidad
Estadounidense    47
Colombiano         5
Dominicano         3
Venezolano         3
Cubano             2
Bahamense          1
Uruguayo           1
Ecuatoriano        1
Canadiense         1
Palestino          1
Brasileño          1
Name: count, dtype: int64

In [45]:
# Eliminar espacios en blanco adelante y atrás en 'NombreCompleto'
df_players['NombreCompleto'] = df_players['NombreCompleto'].str.strip()
df_extranjeros['NombreCompleto'] = df_extranjeros['NombreCompleto'].str.strip()

# Left join para agregar la nacionalidad
df_players = df_players.merge(df_extranjeros, on='NombreCompleto', how='left')
df_players['nacionalidad'] = df_players['nacionalidad'].fillna('Argentino')

In [46]:
df_players['nacionalidad'].value_counts()

nacionalidad
Argentino         312
Estadounidense     47
Colombiano          5
Venezolano          3
Dominicano          3
Cubano              2
Uruguayo            1
Brasileño           1
Ecuatoriano         1
Palestino           1
Bahamense           1
Canadiense          1
Name: count, dtype: int64